### Purpose

The intent of this notebook is to serve as a rapid testing ground for new utilities. Any logic written here should migrate to the src/ directory as proper functions.

#### Import and Constants

In [ ]:
import numpy as np
from pandas.core.generic import NDFrame

# Importing from the local code structure
import os
import sys
module_path = os.path.abspath(os.path.join("..", "src"))
sys.path.append(module_path)
from exercise_log.constants import *
from exercise_log.dataloader import DataLoader
from exercise_log.strength import Exercise
from exercise_log.trend import Trendsetter
from exercise_log.utils import TermColour, convert_mins_to_hour_mins
from exercise_log.vis import plot_resting_heart_rate, plot_strength_over_time, plot_weight, plot_workout_frequency

#### Load Data and Compute Trends

In [ ]:
EXTRAPOLATE_DAYS = 100
N_DAYS_TO_AVG = 8
ROOT_DATA_DIR = "../data"
ROOT_IMG_DIR = "../img"

health_metrics = DataLoader.load_health_metrics(ROOT_DATA_DIR)
travel_days = DataLoader.load_travel_days(ROOT_DATA_DIR)
cardio_workouts = DataLoader.load_cardio_workouts(ROOT_DATA_DIR)
weight_training_workouts = DataLoader.load_weight_training_workouts(ROOT_DATA_DIR)
weight_training_sets = DataLoader.load_weight_training_sets(ROOT_DATA_DIR)
all_workouts = DataLoader.load_all_workouts(cardio_workouts, weight_training_workouts, travel_days)

# n-day average over a week gives a sense of if I'm keeping above a relatively low baseline of 150 minutes/week
n_day_avg_workout_duration = Trendsetter.compute_n_sample_avg(all_workouts, DURATION, N_DAYS_TO_AVG)

# Fit relevant trendlines
weight_trendline = Trendsetter.get_line_of_best_fit(health_metrics, WEIGHT, EXTRAPOLATE_DAYS)
heart_rate_trendline = Trendsetter.get_logarithmic_curve_of_best_fit(health_metrics, RESTING_HEART_RATE, EXTRAPOLATE_DAYS)

In [ ]:
nonnulls = health_metrics[health_metrics[WEIGHT].notnull()]
avg_daily_weightloss = Trendsetter.fit_linear(nonnulls, WEIGHT)[0]
print("Average daily change in weight: {:.2f} lb".format(avg_daily_weightloss))

#### Recent Cardio Workouts

In [ ]:
%%html
<style>.dataframe th { font-size: 10px; } .dataframe td { font-size: 10px; }</style>

In [ ]:
walk_workouts = cardio_workouts[cardio_workouts[WORKOUT_TYPE].isin({WALK_TREADMILL, WALK_OUTDOOR})].copy()
run_workouts = cardio_workouts[cardio_workouts[WORKOUT_TYPE].isin({RUN_TREADMILL})].copy()
bike_workouts = cardio_workouts[cardio_workouts[WORKOUT_TYPE].isin({BIKE_STATIONARY})].copy()
del bike_workouts[STEPS]
del bike_workouts[ELEVATION]

DataLoader.add_computed_cardio_metrics(walk_workouts)
DataLoader.add_computed_cardio_metrics(run_workouts)
display(walk_workouts[-10:])
display(run_workouts[-10:])
display(bike_workouts[-10:])

#### Build Visuals

In [ ]:
# TODO Still need to build at least these visuals:
# * Walking data (max distance, max elevation gain, max duration, pace graph)
# * Strength Metrics grouped by workout
#    - Ideally: drop-down menu to select between various workouts
#    - Need to fine-tune some, remove ones that don't make sense (e.g. that have < 3 days worked)
plot_workout_frequency(all_workouts, n_day_avg_workout_duration, N_DAYS_TO_AVG)
plot_resting_heart_rate(all_workouts, health_metrics, heart_rate_trendline, EXTRAPOLATE_DAYS)
plot_weight(all_workouts, health_metrics, weight_trendline, EXTRAPOLATE_DAYS)

# TODO [Workout frequency graph]
# Change colour of rest days to red or orange and MAYBE also color-code cardio vs weights
# Another idea would be generating sub-graphs for each step of the hierarchy. E.g.
# All -> Cardio -> Walk
# All -> Weight -> Chest (per muscle group, not pairs, bc I don't want to marry the visuals to the current splits)

#### Misc. Metrics

In [ ]:
# Construct a full-picture set of functional fitness metrics
# Will also need to work out a decent way to track progress across so many different metrics, e.g.:
#   - compute and graph aggregate metrics over time
#   - interface that allows drilling-in per-metric and/or group of metrics (e.g. arm strength, walking metrics, etc.)
# Include: 
#   -  Major strength metrics: 1RM deadlift, squat, bench press, strict press
#   2. Cardio metrics:
#        Walking: 1km time, 5km time, 1000ft climb time, max 1-min walking pace
#        Running: 100m sprint, 1 mile time, 5km time, half-marathon time, marathon time, maximum duration
#        Swimming: 400m freestyle time?, 1000m freestyle time?
#        Cardio Health: 2-week average resting heart rate, V02 Max
#   3. Calisthenic metrics: max consecutive push-ups, pull-ups, burpees; static hang duration, vertical leap?
#   4. Minor strength metrics:
#        Chest: N/A
#        Arms: 1RM overhead tricep extension, preacher curl
#        Legs: 1RM leg curl, leg extension, seated calf raise
#        Back: 1RM lawnmowers, lat pulldown
#        Shoulder: 1RM arnold press
#        Forearms: grip strength
#   5. Imbalance measures:
#        Single-arm/leg bicep curl, overhead tricep extension, leg curl, leg extension, etc. vs two-arm/leg
#          Ideally two-arm/leg should be within a few % of 2 * single-arm/leg
#        Relative strength scores
#          within region (e.g. tricep vs. bicep, quads vs. hamstring, back vs. chest)
#          global (e.g. detect outliers such as "weak triceps", compute some total imbalance score)
#   6. Detailed Training Frequency Info
#        Percentage + absolute volume/time of {weight training, calisthenics, cardio}
#        Optionally: include a drill down on compound lifts, isolated lifts, muscle groups, cardio types
#        Optionally: include prescriptive piece to offer a few recommendations for the next workout (e.g. cardio of type X or weight training of type X with a focus on {low, mid, high} reps)
def get_1rm(sets: NDFrame, exercise: str) -> Union[number, str]:
    sets = sets[sets[EXERCISE] == exercise]
    sets = sets[sets[REPS] == 1]
    if sets.empty:
        return "Missing Data"
    return sets[WEIGHT].max()

# Functional Fitness Metric Group 1: Big-Three 1-RM
# TODO modify 1RM to only check sets from the past N-months (3 months maybe?)
one_rm = dict()
for exercise in [Exercise.DEADLIFT, Exercise.SQUATS, Exercise.BENCH_PRESS, Exercise.STRICT_PRESS]:
    one_rm[exercise] = get_1rm(weight_training_sets, exercise)
    print(f"1-RM {exercise}: {int(one_rm[exercise])}")
print(f'Big-Three Combined Weight: {int(one_rm[Exercise.DEADLIFT] + one_rm[Exercise.SQUATS] + one_rm[Exercise.BENCH_PRESS])}')

# Functional Fitness Metric Group 4: Minor Strength Metrics
# TODO same two TODO's as Group 1
print()
for exercise in [
    Exercise.OVERHEAD_TRICEP_EXTENSION, Exercise.PREACHER_CURL,
    Exercise.SINGLE_LEG_LEG_CURL, Exercise.SINGLE_LEG_LEG_EXTENSION, Exercise.BARBELL_CALF_RAISE,
    Exercise.LAWNMOWERS, Exercise.LAT_PULLDOWN,
    Exercise.ARNOLD_PRESS,
]:
    one_rm[exercise] = get_1rm(weight_training_sets, exercise)
    if one_rm[exercise] != "Missing Data":
        one_rm[exercise] = int(one_rm[exercise])
    print(f"1-RM {exercise}: {one_rm[exercise]}")


distance_walked = round(walk_workouts[DISTANCE].to_numpy().sum())
farthest = walk_workouts[DISTANCE].to_numpy().max()
fastest_pace = walk_workouts[PACE].to_numpy().max()
print()
print("Walking Metrics")
print(f"Farthest distance: {farthest}km")
print("Fastest pace (m/s): {:.2f}".format(fastest_pace))
print(f"Total distance: {distance_walked}km")

distance_biked = round(bike_workouts[DISTANCE].to_numpy().sum())
print()
print("Biking Metrics")
print(f"Total distance: {distance_biked}km")

distance_travelled = distance_walked + distance_biked
print()
print("Summary Metrics")
print(f"Total distance travelled: {distance_travelled}km")